In [1]:
import findspark
findspark.init()

In [2]:
import numpy as np

from pyspark.sql import SparkSession
from pyspark.mllib.stat import Statistics

In [3]:
spark = SparkSession.builder\
                    .appName("Assignment1")\
                    .master("local")\
                    .config('spark.executor.memory','32g')\
                    .config('spark.driver.memory','32g')\
                    .config('spark.driver.maxResultSize','2g')\
                    .getOrCreate()

df = spark.read\
    .format("jdbc")\
    .option("url", "jdbc:postgresql://10.222.248.204:5432/mernis")\
    .option("dbtable", "citizen") \
    .option("user", "postgres")\
    .option("password", "postgres")\
    .option("driver", "org.postgresql.Driver") \
    .load()

In [ ]:
# spark = SparkSession.builder\
#                     .appName("Assignment1")\
#                     .master("local")\
#                     .config('spark.executor.memory','32g')\
#                     .config('spark.driver.memory','32g')\
#                     .config('spark.driver.maxResultSize','2g')\
#                     .getOrCreate()

# df = spark.read\
#     .format("jdbc")\
#     .option("url", "jdbc:postgresql://localhost:5432/mernis")\
#     .option("dbtable", "citizen") \
#     .option("user", "postgres")\
#     .option("password", "")\
#     .option("driver", "org.postgresql.Driver") \
#     .load()

root
 |-- uid: long (nullable = true)
 |-- national_identifier: string (nullable = true)
 |-- first: string (nullable = true)
 |-- last: string (nullable = true)
 |-- mother_first: string (nullable = true)
 |-- father_first: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- birth_city: string (nullable = true)
 |-- date_of_birth: string (nullable = true)
 |-- id_registration_city: string (nullable = true)
 |-- id_registration_district: string (nullable = true)
 |-- address_city: string (nullable = true)
 |-- address_district: string (nullable = true)
 |-- address_neighborhood: string (nullable = true)
 |-- street_address: string (nullable = true)
 |-- door_or_entrance_number: string (nullable = true)
 |-- misc: string (nullable = true)



In [4]:
# # Preprocession Convert the type of date_of_birth column into datetime
# from pyspark.sql.functions import to_date
# pattern = 'd/M/y'

# df = df.withColumn('date_of_birth',to_date(df['date_of_birth'],pattern))
# df = df.withColumn('national_identifier',df.national_identifier.cast('long'))


## N6. 计算前10大人口城市人口密度，其中城市的面积可Google搜索，面积单位使用平方千米

In [5]:
# Compute the top-10  city
top10 = df.groupBy('address_city').count().sort("count",ascending=False).limit(10)
top10.rdd.keys().collect()

['ISTANBUL',
 'ANKARA',
 'IZMIR',
 'BURSA',
 'AYDIN',
 'ADANA',
 'KONYA',
 'ANTALYA',
 'MERSIN',
 'KOCAELI']

In [6]:
# Results from DB-city.com：
# 'ISTANBUL', city/province 5343 
#  'ANKARA', city 2516 province 17501
#  'IZMIR', city 6412.92 province 12473
#  'BURSA', city 2259.35 province 10422
#  'AYDIN', city 627 province 8187
#  'ADANA', city 3708.19 province 13834
#  'KONYA', city 6617 province 39390
#  'ANTALYA', city 2099 province 22111
#  'MERSIN', city 3038 province 17661
#  'KOCAELI', city/province 3676.20 

# Corresponding areas
city_area = {
 'ISTANBUL':5343,
 'ANKARA':17501,
 'IZMIR':12473,
 'BURSA':10422,
 'AYDIN':8187,
 'ADANA':13834,
 'KONYA':39390,
 'ANTALYA':22111,
 'MERSIN':17661,
 'KOCAELI':3676.20,
}

In [7]:
# Compute the density of the population
def density(kv):
    return (kv[0], kv[1]/city_area[kv[0]])
    
ans = top10.select('address_city','count').rdd
ans.map(density).collect()

[('ISTANBUL', 1652.5517499532098),
 ('ANKARA', 176.07239586309353),
 ('IZMIR', 223.79531788663513),
 ('BURSA', 171.22155056611015),
 ('AYDIN', 172.42335409796019),
 ('ADANA', 100.51301142113633),
 ('KONYA', 33.82178217821782),
 ('ANTALYA', 58.327845868572204),
 ('MERSIN', 62.22541192457958),
 ('KOCAELI', 277.0678961971601)]

## N7. 根据人口的出身地和居住地，分别统计土耳其跨行政区流动人口和跨城市流动人口占总人口的比例。

In [8]:
# Cross-city mobility: One whose id_registration_city and address_city are different
# Cross-district mobiilty: One whose id_registration_district and address_district are different 

In [4]:
df7 = df.select('id_registration_city','id_registration_district','address_city','address_district')

In [5]:
def crossMan(x):
    for i in range(0,4):
        if x[i] is None:
            return _,(0,0,0)
    crossCity = 0
    crossDist = 0
    # Compare id_registration_city and address_city
    if x[0] != x[2]:
        crossCity = 1
    if x[1] != x[3]:
        crossDist = 1
    return _,(1, crossCity, crossDist)

In [6]:
ans = df7.rdd.map(crossMan).reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1],  x[2]+y[2])).mapValues(lambda x: (x[1]/x[0], x[2]/x[0]))                   

In [8]:
ans.values().collect()

[(0.36136473347451104, 0.523862199546482)]

## H1. 某人所在城市的预测模型:给定一个人的所有信息(除了所在城市)，预测这个人所在的城市。 分析该模型Top1到 Top 5的预测准确度

In [10]:
df.printSchema()

root
 |-- uid: long (nullable = true)
 |-- national_identifier: string (nullable = true)
 |-- first: string (nullable = true)
 |-- last: string (nullable = true)
 |-- mother_first: string (nullable = true)
 |-- father_first: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- birth_city: string (nullable = true)
 |-- date_of_birth: string (nullable = true)
 |-- id_registration_city: string (nullable = true)
 |-- id_registration_district: string (nullable = true)
 |-- address_city: string (nullable = true)
 |-- address_district: string (nullable = true)
 |-- address_neighborhood: string (nullable = true)
 |-- street_address: string (nullable = true)
 |-- door_or_entrance_number: string (nullable = true)
 |-- misc: string (nullable = true)



In [ ]:
#######
# SQL查询 select COUNT(*) from citizen where birth_city = '<NULL>'
# 结果显示26111条数据为空 共计49611709
#  select distinct birth_city from citizen
# 显示很多无效数据如图
# 姑数据清洗
#####

In [ ]:
df1 = df.filter("birth_city LIKE '^[A-Z]' ")

In [ ]:
# Feature Extraction
# df1 = births.withColumn('BIRTH_PLACE_INT', births['BIRTH_PLACE']\
#     .cast(typ.IntegerType()))


label = 'address_city'
stringIndexer = StringIndexer(inputCol=label, outputCol="indexed")
model = stringIndexer.fit(df1)
df1 = model.transform(df1)

In [ ]:
# Split the dataset
train_dat, valid_dat, test_dat = df1.randomSplit([0.7, 0.1, 0.2])

In [17]:
from pyspark.ml.classification import NaiveBayes

<function pyspark.ml.param.Params.explainParams(self)>

In [ ]:
# Logistic Regressioin


In [ ]:
# Decision Tree/Random Forest？
